# Modeling: Random forest

In [ ]:
#Installing requirements
#!pip install -r requirements.txt

In [ ]:
#Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import os
from collections import Counter
import statistics
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import random
import pickle

from pylab import rcParams
from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#Loading the csv file
import pandas as pd
df=pd.read_csv('/datasets/gdrive/DS4A - Team Project/data/all_data.csv', sep=',', encoding="latin-1")

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/gdrive/DS4A - Team Project/data/all_data.csv'

In [ ]:
low = pickle.load(open("variables/df.dat", "rb"))

In [ ]:
bow_counts = pickle.load(open("variables/bow_counts.dat", "rb"))
bow_counts

CountVectorizer(stop_words=['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los',
                            'del', 'se', 'las', 'por', 'un', 'para', 'con',
                            'no', 'una', 'su', 'al', 'lo', 'como', 'más',
                            'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí',
                            'porque', ...],
                tokenizer=<function word_tokenize at 0x7f51df1adcb0>)

# Random Forest

In [ ]:
#One stratified train-test split (80%-20%)

reviews_train, reviews_test = train_test_split(low, test_size=0.2, random_state=42, stratify=low['TIPO DE PQRSDF'])
X_train_bow = bow_counts.fit_transform(reviews_train['TEXTO PQRSDF'])
X_test_bow = bow_counts.transform(reviews_test['TEXTO PQRSDF'])
y_train_bow = reviews_train['TIPO DE PQRSDF']
y_test_bow = reviews_test['TIPO DE PQRSDF']


In [ ]:
# Instantiate the estimator
rf_model_all = RandomForestClassifier(n_estimators=100, random_state=42)

# Training the data
rf_model_all.fit(X_train_bow, y_train_bow)

# Generating predictions
test_pred_rf_prob = rf_model_all.predict_proba(X_test_bow)
test_pred_rf_all = rf_model_all.predict(X_test_bow)

In [ ]:
#Metrics
print("F1 score: ", f1_score(y_test_bow,test_pred_rf_all,average='micro'))
print("Accuracy: ", accuracy_score(y_test_bow,test_pred_rf_all)* 100)

print(confusion_matrix(y_test_bow, test_pred_rf_all))

F1 score:  0.6297709923664122
Accuracy:  62.97709923664122
[[59  9 32]
 [10 41  9]
 [26 11 65]]


**Hyperparameter tuning and cross-validation**

In [ ]:
#to create a parameter grid to sample from during fitting we import RandomizedSearchCV:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [ ]:
# We instantiate the RandomizedSearchCV and fit it like any Scikit-Learn model:
# Use the random grid to search for best hyperparameters

# First we create the base model to tune
rf = RandomForestClassifier(random_state=42)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(X_train_bow, y_train_bow)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [ ]:
# View the best parameters from fitting the random search and the best score (mean cross-validated accuracy):
print("Tuned Random Forest best Parameters: {}".format(rf_random.best_params_))
print("Tuned and cross-validated Random Forest Accuracy: {}".format(rf_random.best_score_))

# Get the predictions on the test set
y_pred = rf_random.predict(X_test_bow)

print("F1 score when predicting on the test set: ", f1_score(y_test_bow,y_pred,average='micro'))
print("Accuracy when predicting on the test set: ", accuracy_score(y_test_bow,y_pred)* 100)

Tuned Random Forest best Parameters: {'n_estimators': 400, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 90, 'bootstrap': False}
Tuned and cross-validated Random Forest Accuracy: 0.6804142102778602
F1 score when predicting on the test set:  0.6717557251908397
Accuracy when predicting on the test set:  67.17557251908397


In [ ]:
print(confusion_matrix(y_test_bow, y_pred))

[[67 10 23]
 [ 6 46  8]
 [28 11 63]]


In [ ]:
#Predictor tool
pqrs=[]
num=int(input('Ingrese cuantas PQRS quiere ingresar'))
for i in range(num):
    inp=input('Ingrese su PQRS')
    pqrs.append(inp)

pqrs = pd.DataFrame(pqrs, columns=['TEXTO PQRSDF'])
#Transform 
data=bow_counts.transform(pqrs['TEXTO PQRSDF'])

#Predicting the output
prob= rf_random.predict_proba(data) 
pred = rf_random.predict(data)

#Assign the data to the missing
pqrs['TIPO DE PQRSDF'] = pred

pqrs

,TEXTO PQRSDF,TIPO DE PQRSDF
0,Solicito información sobre vacunación COVID.,P


**Feature importances** 

In [ ]:
# Let's fit a model with the best parameters, no cross-validation, so we can see the feature importances
rf_best_model = RandomForestClassifier(n_estimators=1400, random_state=42, min_samples_split=2, min_samples_leaf=2, max_features='auto', max_depth=70, bootstrap=True)
rf_best_model.fit(X_train_bow, y_train_bow)

RandomForestClassifier(max_depth=70, min_samples_leaf=2, n_estimators=1400,
                       random_state=42)

In [ ]:
# Let's get the n-grams which were most important for the predictions. For this we use rf_best_model,
# because the class RandomSearchCV does not have feature_importances_ as an attribute.
feature_importances = pd.DataFrame(
    rf_best_model.feature_importances_,
    index=bow_counts.get_feature_names(),
    columns=['importance']
)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
feature_importances.sort_values(['importance'], ascending=False).head(17)

,importance
busca,0.070131
inconformidad,0.063112
manifiesta,0.054122
informacion,0.043597
solicita,0.026106
medicamentos,0.020318
parte,0.018963
2021,0.016730
asignada,0.016457
entrega,0.013986


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=746f9757-a4f8-490f-a84c-875a1acb7471' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>